## NBA 2017

**Scenarios**

The data set contains aggregate individual statistics for 67 NBA seasons. From basic box-score attributes such as points, assists, rebounds etc., to more advanced money-ball like features such as Value Over Replacement.

**Pre-Processing**

In this task not all data will be used, only data in 2017. So it is necessary to do filtering at the beginning. Besides that there are some players who make team transfers in the NBA transfer market so that there is duplication of playerdata. Therefore we can use the df.drop_duplicates() syntax to solve this. Delete columns that have as many missing values​​as the entire row of data. Then you can do additional pre-processing if needed or you can immediately process the data.

**Some goals of this project:**
1. Who is the youngest and oldest player in the NBA in 2017 for each team
2. Which player has the most minutes played (MP) in each position (Pos)?
3. Which team has the highest average total rebound percentage (TRB%), assist percentage(AST%), steal percentage (STL%), and block percentage (BLK%)?
4. Who is the best player in your opinion based on his record stats? note: you can refer tovariables point (PTS), assists, rebounds, or anything else. A combination of several variableswould be nice.
5. Which team has the best average stat record of their players? Note: you can refer to points,assists, rebounds, or anything else. A combination of several variables would be nice

###Pre-processing

In [ ]:
# Import library
import pandas as pd
import numpy as np
from fancyimpute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [ ]:
# Import the dataset
df_raw = pd.read_csv('Seasons_Stats.csv')
df_raw.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,NaN,0.467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,3.6,3.5,NaN,NaN,NaN,NaN,NaN,NaN,144.0,516.0,0.279,NaN,NaN,NaN,144.0,516.0,0.279,0.279,170.0,241.0,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,NaN,0.387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,0.6,2.2,NaN,NaN,NaN,NaN,NaN,NaN,102.0,274.0,0.372,NaN,NaN,NaN,102.0,274.0,0.372,0.372,75.0,106.0,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,NaN,0.259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,2.8,3.6,NaN,NaN,NaN,NaN,NaN,NaN,174.0,499.0,0.349,NaN,NaN,NaN,174.0,499.0,0.349,0.349,90.0,129.0,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,NaN,0.395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN,NaN,22.0,86.0,0.256,NaN,NaN,NaN,22.0,86.0,0.256,0.256,19.0,34.0,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,NaN,0.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN,NaN,21.0,82.0,0.256,NaN,NaN,NaN,21.0,82.0,0.256,0.256,17.0,31.0,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [ ]:
# Check missing data and data type
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24691 entries, 0 to 24690
Data columns (total 53 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  24691 non-null  int64  
 1   Year        24624 non-null  float64
 2   Player      24624 non-null  object 
 3   Pos         24624 non-null  object 
 4   Age         24616 non-null  float64
 5   Tm          24624 non-null  object 
 6   G           24624 non-null  float64
 7   GS          18233 non-null  float64
 8   MP          24138 non-null  float64
 9   PER         24101 non-null  float64
 10  TS%         24538 non-null  float64
 11  3PAr        18839 non-null  float64
 12  FTr         24525 non-null  float64
 13  ORB%        20792 non-null  float64
 14  DRB%        20792 non-null  float64
 15  TRB%        21571 non-null  float64
 16  AST%        22555 non-null  float64
 17  STL%        20792 non-null  float64
 18  BLK%        20792 non-null  float64
 19  TOV%        19582 non-nul

In [ ]:
# Filter 2017 Data
df_raw2 = df_raw.loc[df_raw['Year'] == 2017]

# Drop duplicate
df_raw3 = df_raw2.drop_duplicates(subset=['Player'])

# Drop Unsignificant Column
df_raw4 = df_raw3.drop(columns=['Unnamed: 0', 'blanl', 'blank2'])

# Check missing data and data type to make sure
df_raw4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 24096 to 24690
Data columns (total 50 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    486 non-null    float64
 1   Player  486 non-null    object 
 2   Pos     486 non-null    object 
 3   Age     486 non-null    float64
 4   Tm      486 non-null    object 
 5   G       486 non-null    float64
 6   GS      486 non-null    float64
 7   MP      486 non-null    float64
 8   PER     486 non-null    float64
 9   TS%     485 non-null    float64
 10  3PAr    485 non-null    float64
 11  FTr     485 non-null    float64
 12  ORB%    486 non-null    float64
 13  DRB%    486 non-null    float64
 14  TRB%    486 non-null    float64
 15  AST%    486 non-null    float64
 16  STL%    486 non-null    float64
 17  BLK%    486 non-null    float64
 18  TOV%    485 non-null    float64
 19  USG%    486 non-null    float64
 20  OWS     486 non-null    float64
 21  DWS     486 non-null    float64
 

There are some missing values in the data. The column that have missing values are TS%, 3PAr, FTr, TOV%, FG%, 3P%, 2P%, eFG%, and FT%. These missing values need to be imputed so we can analyze the data more.

**Data Imputation**

In [ ]:
# Groupby the data based on its type
num_data = df_raw4.select_dtypes(include=[np.number]) #for numerical data
cat_data = df_raw4.select_dtypes(exclude=[np.number]) #for categorical data

In [ ]:
# State the imputer method, we use MICE(Multiple Imputation by Chained Equations)
MICE_imputer = IterativeImputer()

# Impute the numerical data, the categorical data is not imputed because there are no missing values
num_data_MICE = num_data.copy(deep=True)
num_data_MICE. iloc[: , :] = MICE_imputer. fit_transform(num_data_MICE)

In [ ]:
# Merge the imputed data and the categorical data
df = pd.concat([cat_data, num_data_MICE], axis=1, sort=False)

# Assign the correct data type
df['Age'] = df['Age'].astype(int)
df['Year'] = df['Year'].astype(int)

# Check the final dataset
df.head()

,Player,Pos,Tm,Year,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
24096,Alex Abrines,SG,OKC,2017,23,68.0,6.0,1055.0,10.1,0.560,0.724,0.144,1.9,7.1,4.5,5.5,1.7,0.6,8.3,15.9,1.2,0.9,2.1,0.095,-0.3,-2.2,-2.5,-0.1,134.0,341.0,0.393,94.0,247.0,0.381,40.0,94.0,0.426,0.531,44.0,49.0,0.898,18.0,68.0,86.0,40.0,37.0,8.0,33.0,114.0,406.0
24097,Quincy Acy,PF,TOT,2017,26,38.0,1.0,558.0,11.8,0.565,0.529,0.353,3.9,18.0,11.0,4.9,1.2,2.0,9.7,16.8,0.5,0.5,0.9,0.082,-1.8,-1.2,-3.0,-0.1,70.0,170.0,0.412,37.0,90.0,0.411,33.0,80.0,0.413,0.521,45.0,60.0,0.750,20.0,95.0,115.0,18.0,14.0,15.0,21.0,67.0,222.0
24100,Steven Adams,C,OKC,2017,23,80.0,80.0,2389.0,16.5,0.589,0.002,0.392,13.0,15.5,14.2,5.4,1.8,2.6,16.0,16.2,3.3,3.1,6.4,0.130,-0.7,1.2,0.5,1.5,374.0,655.0,0.571,0.0,1.0,0.000,374.0,654.0,0.572,0.571,157.0,257.0,0.611,282.0,333.0,615.0,86.0,88.0,78.0,146.0,195.0,905.0
24101,Arron Afflalo,SG,SAC,2017,31,61.0,45.0,1580.0,9.0,0.559,0.360,0.221,0.7,8.4,4.6,7.4,0.7,0.4,8.4,14.4,1.2,0.2,1.4,0.043,-1.4,-2.0,-3.5,-0.6,185.0,420.0,0.440,62.0,151.0,0.411,123.0,269.0,0.457,0.514,83.0,93.0,0.892,9.0,116.0,125.0,78.0,21.0,7.0,42.0,104.0,515.0
24102,Alexis Ajinca,C,NOP,2017,28,39.0,15.0,584.0,12.9,0.529,0.022,0.225,8.3,23.8,16.0,3.1,1.7,3.1,13.7,17.2,0.0,0.9,1.0,0.080,-5.1,1.0,-4.1,-0.3,89.0,178.0,0.500,0.0,4.0,0.000,89.0,174.0,0.511,0.500,29.0,40.0,0.725,46.0,131.0,177.0,12.0,20.0,22.0,31.0,77.0,207.0


### 1.Who is the youngest and oldest player in the NBA in 2017 for each team (Tm) ?

In [ ]:
#Youngest Player Each Team
young_array = (df.groupby('Tm')['Tm', 'Age', 'Player'].min()).to_numpy()
young = pd.DataFrame(young_array, columns=[['Team', 'Youngest Age', 'Youngest Player']])

#Oldest Player Each Team
old_array = (df.groupby('Tm')['Age', 'Player'].max()).to_numpy()
old = pd.DataFrame(old_array, columns=[['Oldest Age', 'Oldest Player']])

In [ ]:
#Merge into one dataframe
young_old = pd.concat([young, old], axis=1, sort=False)
young_old.drop(columns='Team')
young_old

,Team,Youngest Age,Youngest Player,Oldest Age,Oldest Player
0,ATL,22,DeAndre' Bembry,32,Tim Hardaway
1,BOS,20,Al Horford,31,Tyler Zeller
2,BRK,21,Anthony Bennett,36,Trevor Booker
3,CHI,21,Bobby Portis,35,Robin Lopez
4,CHO,21,Aaron Harrison,31,Treveon Graham
5,CLE,21,Channing Frye,38,Tristan Thompson
6,DAL,21,A.J. Hammons,38,Wesley Matthews
7,DEN,19,Alonzo Gee,36,Wilson Chandler
8,DET,20,Andre Drummond,34,Tobias Harris
9,GSW,20,Anderson Varejao,36,Zaza Pachulia


### 2.Which player has the most minutes played (MP) in each position (Pos)?

In [ ]:
#Player with Most Minutes Played Each Pos
df.groupby('Pos')['MP', 'Player'].max()

,MP,Player
Pos,,
C,3030.0,Zaza Pachulia
PF,2803.0,Zach Randolph
PF-C,980.0,Joffrey Lauvergne
PG,2947.0,Yogi Ferrell
SF,3048.0,Wilson Chandler
SG,2796.0,Zach LaVine


### 3.Which team has the highest average total rebound percentage (TRB%), assist percentage(AST%), steal percentage (STL%), and block percentage (BLK%)?

In [ ]:
# Team with Highest TRB, AST, STL, BLK
TRB = df.groupby('Tm')['TRB%'].mean()
AST = df.groupby('Tm')['AST%'].mean()
STL = df.groupby('Tm')['STL%'].mean()
BLK = df.groupby('Tm')['BLK%'].mean()

# Print Team with Highest TRB, AST, STL, BLK
print('Highest Average Total Rebound Percentage: ',TRB.idxmax(), 'with average of', round(TRB.max(),2))
print('Highest Average Assist Percentage: ',AST.idxmax(), 'with average of', round(AST.max(),2))
print('Highest Average Steal Percentage: ',STL.idxmax(), 'with average of', round(STL.max(),2))
print('Highest Average Block Percentage: ',BLK.idxmax(), 'with average of', round(BLK.max(),2))

Highest Average Total Rebound Percentage:  WAS with average of 13.45
Highest Average Assist Percentage:  DEN with average of 15.86
Highest Average Steal Percentage:  MIN with average of 2.37
Highest Average Block Percentage:  GSW with average of 2.74


### 4.Who is the best player in your opinion based on his record stats? note: you can refer to variables point (PTS), assists, rebounds, or anything else. A combination of several variables would be nice.

I’ll explain what I will use as features and some simple feature selection processes.

***PER*** stands for Player Efficiency Rating, and is basically a calculation of all positives and negatives simple stats.

***BPM*** stands for Box Plus Minus and is an advanced statistic for evaluating players’ quality and contribution to the team. It takes more team based stats into the calculation in comparison to PER.

***TS%*** stands for True Shooting Percentage, and calculation is rather simple (and actually included in stats above)

***USG%*** is short for Usage Percentage and is an estimate of team plays which a player uses while on the court.

And in the end, *WS and WS/48* stand for Win Shares and Win shares per 48 minutes. That stat attempts to divide team success on individual members of the team.

In [ ]:
# Create new dataframe that include the scoring criteria
df_best = df[['Player', 'Tm', 'Pos','PER', 'BPM', 'TS%', 'USG%', 'WS', 'WS/48']]
df_best = df_best.reset_index(drop=True)
df_best.head()

,Player,Tm,Pos,PER,BPM,TS%,USG%,WS,WS/48
0,Alex Abrines,OKC,SG,10.1,-2.5,0.560,15.9,2.1,0.095
1,Quincy Acy,TOT,PF,11.8,-3.0,0.565,16.8,0.9,0.082
2,Steven Adams,OKC,C,16.5,0.5,0.589,16.2,6.4,0.130
3,Arron Afflalo,SAC,SG,9.0,-3.5,0.559,14.4,1.4,0.043
4,Alexis Ajinca,NOP,C,12.9,-4.1,0.529,17.2,1.0,0.080


In [ ]:
# Declare standardization method
scaler = MinMaxScaler()

# Groupby the data based on its type
df_best_num = df_best.select_dtypes(include=[np.number]) #for numerical data
df_best_cat = df_best.select_dtypes(exclude=[np.number]) #for categorical data

# Transform the data
scaler.fit(df_best_num)
score = pd.DataFrame(np.sum((scaler.transform(df_best_num)), axis=1), columns=['score'])

#Finalize and check the data
df_best_final = pd.concat([df_best_cat, score], axis=1, sort=False)
df_best_final.head()

,Player,Tm,Pos,score
0,Alex Abrines,OKC,SG,3.000000
1,Quincy Acy,TOT,PF,2.961109
2,Steven Adams,OKC,C,3.553302
3,Arron Afflalo,SAC,SG,2.817977
4,Alexis Ajinca,NOP,C,2.926396


In [ ]:
#Print Best Player
print('Best NBA Player in 2017 is:', '\n', df_best_final.sort_values(by='score',ascending=False).iloc[0])

Best NBA Player in 2017 is: 
 Player    Russell Westbrook
Tm                      OKC
Pos                      PG
score               5.28616
Name: 457, dtype: object


### 5.Which team has the best average stat record of their players? Note: you can refer to points,assists, rebounds, or anything else. A combination of several variables would be nice

In [ ]:
# We use the data we create from the previous question
best_team = df_best_final.groupby('Tm')['score'].mean()

# Check the data
best_team.head()

Tm
ATL    2.913775
BOS    3.299916
BRK    3.032249
CHI    3.038595
CHO    3.026816
Name: score, dtype: float64

In [ ]:
# Print the best team with best average stat record of their players
print('Best NBA Team in 2017 is: ',best_team.idxmax(), 'with average score of', round(best_team.max(),2))

Best NBA Team in 2017 is:  GSW with average score of 3.45
